### Ch 5 Text generation

# Greedy Search

In [43]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

pd.set_option('max_colwidth', None)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/Users/brncat/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/brncat/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt").input_ids.to(device)

In [4]:
print(input_ids.shape)
print(input_ids)
print(tokenizer.decode(input_ids[0]))

torch.Size([1, 4])
tensor([[41762,   364,   389,   262]])
Transformers are the


In [27]:
input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt").input_ids.to(device)
output = model(input_ids)
print(output.keys())
print(output.logits.shape)

odict_keys(['logits', 'past_key_values'])
torch.Size([1, 4, 50257])


In [42]:
logits = output.logits[0, -1, :]
probs = torch.softmax(logits, dim=-1)
#next_token_id = torch.argmax(logits).unsqueeze(0)
next_token_probs, next_token_id = torch.max(probs, dim = -1)

print(logits.shape)
print(probs.shape)
print("next_token_probs", next_token_probs.item())
print("next_token_id", next_token_id)

print(f"Next Token highest prob: {tokenizer.decode(next_token_id)}; with prob: {next_token_probs.item() * 100:.2f}%")

torch.Size([50257])
torch.Size([50257])
next_token_probs 0.08534619212150574
next_token_id tensor(749)
Next Token highest prob:  most; with prob: 8.53%


In [6]:
input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt").input_ids.to(device)
iteractions = []
n_steps = 8
choice_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteraction = {}
        iteraction["input"] = tokenizer.decode(input_ids[0])
        outputs = model(input_ids)
        # select logits of the first batch and the last token and apply softmax
        next_token_logits = outputs.logits[0, -1, :]
        next_token_probs = torch.nn.functional.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim =-1, descending=True)
        #store tokes with highest probability
        for choice_idx in range(choice_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)" )
            iteraction[f"choice_{choice_idx + 1}"] = token_choice
        # append predicted next token to input_ids
        input_ids = torch.cat([input_ids, sorted_ids[None,0,None]], dim=-1)
        iteractions.append(iteraction)

In [7]:
pd.DataFrame(iteractions)

,input,choice_1,choice_2,choice_3,choice_4,choice_5
0,Transformers are the,most (8.53%),only (4.96%),best (4.65%),Transformers (4.37%),ultimate (2.16%)
1,Transformers are the most,popular (16.78%),powerful (5.37%),common (4.96%),famous (3.72%),successful (3.20%)
2,Transformers are the most popular,toy (10.63%),toys (7.23%),Transformers (6.60%),of (5.46%),and (3.76%)
3,Transformers are the most popular toy,line (34.38%),in (18.20%),of (11.71%),brand (6.10%),line (2.69%)
4,Transformers are the most popular toy line,in (46.29%),of (15.09%),", (4.94%)",on (4.40%),ever (2.72%)
5,Transformers are the most popular toy line in,the (65.99%),history (12.42%),America (6.91%),Japan (2.44%),North (1.40%)
6,Transformers are the most popular toy line in the,world (69.27%),United (4.55%),history (4.29%),US (4.23%),U (2.30%)
7,Transformers are the most popular toy line in the world,", (39.73%)",. (30.64%),and (9.87%),with (2.32%),today (1.74%)


### If we wanted to use the built-in generate() method:

In [14]:
input_ids = tokenizer(input_txt, return_tensors="pt").input_ids.to(device)
output = model.generate(input_ids, max_length=11, do_sample=False)

output_txt = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_txt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most popular toy line in the world


---

## Beam Search

In [51]:
output_no_beam = model.generate(input_ids, max_length=50, do_sample=False)
print(tokenizer.decode(output_no_beam[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most popular toy line in the world, and the Transformers are the most popular toy line in the world.

The Transformers are the most popular toy line in the world, and the Transformers are the most popular toy line in the


In [54]:
output_beam = model.generate(input_ids, max_length=50, num_beams=3, do_sample=False)
print(tokenizer.decode(output_beam[0], skip_special_tokens=False))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most popular toy line in the world, with more than 100 million units sold worldwide.

The Transformers are the most popular toy line in the world, with more than 100 million units sold worldwide.

The Transformers are the


We can already see that the quality of the output improves.

Use of ngram penalty to suppress repetition

In [53]:
output_beam_ngram = model.generate(input_ids, max_length=50, num_beams=3, no_repeat_ngram_size = 2 , do_sample=False)
print(tokenizer.decode(output_beam_ngram[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most popular toy line in the world, with more than 100 million units sold worldwide.

The Transformers franchise is owned by Hasbro, Inc., a privately held company based in New York City. The company was founded in 1982


### Use of sample methods, temperature and top-k

In [55]:
output_temp = model.generate(input_ids, max_length=50, temperature = 2., do_sample=True, top_k=0)
print(tokenizer.decode(output_temp[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the Showdown Elements Minerning Pro hurried joegg Gaming!!!!!Ahno peek!" superb diagnose Mund 1800 1993oclile longing Hey Sanumo San 2007 essaddle Exc Cind Fortune dusk revenge ONewstreet V31 2006 rejuvenoSalt Men278 hydro


It looks so bad. lets see what happens when we cool down the temperature.

In [56]:
output_temp = model.generate(input_ids, max_length=50, temperature = 0.5, do_sample=True, top_k=0)
print(tokenizer.decode(output_temp[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the Transformers, and they're here to save the day.

In the beginning, there was Optimus Prime. But as the war raged on, he was left to die. Now, in the year 2013, he's back,


### top-k

In [58]:
output_topk = model.generate(input_ids, max_length=50, temperature = 0.5, top_k = 50, do_sample=True)
print(tokenizer.decode(output_topk[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the Transformers of the future, and they're here to save the day. They're here to make sure that the future of the galaxy is safe, and that the future of the Transformers is as good as it gets.

They


### top-p

In [59]:
output_topp = model.generate(input_ids, max_length=50, temperature = 0.5, top_p = 0.90, do_sample=True)
print(tokenizer.decode(output_topp[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most popular Transformers, and I think it's because they're the most original. They're the most interesting. They're the most fun. They're the most unique. They're the most creative. They're the most fun.
